In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.environment.models.stock import StockBuffer
from apps.environment_simulator.sevice_layer.stock_simulator import StockSimulator
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q
from os import path, environ
from apps.environment.service_layer.data_transformer import DataTransformer

In [ ]:



nwb_starting_time_step = datetime(year=2023, month=6, day=1)
now_time = pytz.utc.localize(datetime.now())

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
stck_buffs = nwb_dfs[2]
# rcrds = stck_buffs.to_dict(orient='records')



cf.go_offline()
cf.set_config_file(world_readable=True, theme="white")

stck_buffs['captured_at'] = pd.to_datetime(stck_buffs['captured_at'])

# Set 'captured_at' as the index
stck_buffs.set_index('captured_at', inplace=True)

# Group by 'index' and plot the 'price_snapshot' for each group
grouped_df = stck_buffs.groupby('index')['price_snapshot']

# Plot each group separately
cf.go_offline()

for stock_index, price_snapshot_group in grouped_df:
    price_snapshot_group.iplot(
        kind='line',
        xTitle='Captured At',
        yTitle='Price Snapshot',
        title=f'Price Snapshot Over Time - Stock Index {stock_index}',
        legend=True
    )

# time_values = []
# price_variations = {}
# for snapshot in rcrds:
#     inx = snapshot["index"]
#     if snapshot["index"] not in price_variations:
#         price_variations[inx] = []
#         time_values.append(snapshot["captured_at"])
#     price_variations[inx].append(snapshot["price_snapshot"])

# # print(price_variations)
# for stck_name in price_variations:
#     price_snaps = price_variations[stck_name]
#     price_array = np.array(price_snaps)
#     max_price = max(price_array)
#     normalized_snaps = price_array/max_price
#     price_variations[stck_name] = normalized_snaps

    

# stock_names = list(price_variations.keys())
# time_values = pd.to_datetime(time_values)

# # print(price_variations)
# df_regret_comparisons = pd.DataFrame(price_variations,)
# df_regret_comparisons.iplot(
#     xTitle='time_steps',
#     yTitle='Normalized Prices'
# )